<a href="https://colab.research.google.com/github/eunzzae/FC_Project_essay/blob/main/ML%EC%BA%90%EA%B8%80%ED%94%8C%EC%A0%9D_%EC%97%90%EC%84%B8%EC%9D%B4%ED%92%88%EC%A7%88%EC%98%88%EC%B8%A1_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%8C%8C%EC%95%85_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
%matplotlib inline

data_path = '/content/drive/MyDrive/Colab Notebooks/data/essay_quality/'

train_l = pd.read_csv(data_path+'train_logs.csv')
train_s = pd.read_csv(data_path+'train_scores.csv')
test = pd.read_csv(data_path+'test_logs.csv')
submission = pd.read_csv(data_path+'sample_submission.csv')

In [ ]:
train_l.info()
train_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405898 entries, 0 to 8405897
Data columns (total 11 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               object
 1   event_id         int64 
 2   down_time        int64 
 3   up_time          int64 
 4   action_time      int64 
 5   activity         object
 6   down_event       object
 7   up_event         object
 8   text_change      object
 9   cursor_position  int64 
 10  word_count       int64 
dtypes: int64(6), object(5)
memory usage: 705.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2471 entries, 0 to 2470
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      2471 non-null   object 
 1   score   2471 non-null   float64
dtypes: float64(1), object(1)
memory usage: 38.7+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               6 non-null      object
 1   event_id         6 non-null      int64 
 2   down_time        6 non-null      int64 
 3   up_time          6 non-null      int64 
 4   action_time      6 non-null      int64 
 5   activity         6 non-null      object
 6   down_event       6 non-null      object
 7   up_event         6 non-null      object
 8   text_change      6 non-null      object
 9   cursor_position  6 non-null      int64 
 10  word_count       6 non-null      int64 
dtypes: int64(6), object(5)
memory usage: 656.0+ bytes


In [ ]:
train_l.describe(include='all')

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
count,8405898,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8405898,8405898,8405898,8405898,8.405898e+06,8.405898e+06
unique,2471,NaN,NaN,NaN,NaN,50,131,130,4111,NaN,NaN
top,d3fef198,NaN,NaN,NaN,NaN,Input,q,q,q,NaN,NaN
freq,12876,NaN,NaN,NaN,NaN,6726796,5365319,5364789,6128208,NaN,NaN
mean,NaN,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,NaN,NaN,NaN,NaN,1.222964e+03,2.314687e+02
std,NaN,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,NaN,NaN,NaN,NaN,9.485242e+02,1.759088e+02
min,NaN,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
25%,NaN,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,NaN,NaN,NaN,NaN,4.990000e+02,9.600000e+01
50%,NaN,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,NaN,NaN,NaN,NaN,1.043000e+03,2.000000e+02
75%,NaN,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,NaN,NaN,NaN,NaN,1.706000e+03,3.270000e+02


In [ ]:
# 범주형 데이터 인코딩

# activity를 정수로 변환하는 함수
def map_act_code(i_activity):
    result = -1
    if i_activity == 'Input':
        result = 1
    elif i_activity == 'Remove/Cut':
        result = 2
    elif i_activity == 'Nonproduction':
        result = 3
    elif i_activity == 'Replace':
        result = 4
    elif i_activity == 'Paste':
        result = 5
    else:
        result = 0

    return result

train_l['activity_num'] = train_l['activity'].apply(map_act_code)
print(train_l.activity_num.value_counts())
train_l.head()

1    6726796
2     970158
3     703851
4       4448
5        599
0         46
Name: activity_num, dtype: int64


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,activity_num
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0,3
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1,1


In [ ]:
# 범주형 피처
print(train_l.activity.value_counts())
print("-"*20)
print(train_l.down_event.value_counts())
print("-"*20)
print(train_l.up_event.value_counts())
print("-"*20)
print(train_l.text_change.value_counts())

Input                                     6726796
Remove/Cut                                 970158
Nonproduction                              703851
Replace                                      4448
Paste                                         599
Move From [1306, 1371] To [1061, 1126]          2
Move From [13, 65] To [9, 61]                   1
Move From [274, 314] To [299, 339]              1
Move From [624, 625] To [845, 846]              1
Move From [1861, 2063] To [1766, 1968]          1
Move From [1766, 1968] To [1861, 2063]          1
Move From [2091, 2179] To [252, 340]            1
Move From [923, 1077] To [340, 494]             1
Move From [0, 1] To [590, 591]                  1
Move From [999, 1000] To [1000, 1001]           1
Move From [0, 75] To [1, 76]                    1
Move From [1651, 1769] To [1565, 1683]          1
Move From [61, 136] To [0, 75]                  1
Move From [623, 632] To [624, 633]              1
Move From [75, 134] To [304, 363]               1


In [ ]:
def resumetable(df):
  print(f'데이터셋 형상: {df.shape}')
  summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
  summary = summary.reset_index()
  summary = summary.rename(columns={'index':'피처'})
  summary['결측값 개수'] = df.isnull().sum().value_counts
  summary['고윳값 개수'] = df.nunique().values

  # 고윳값 한번에 출력하기
  unique_feature = []
  for i in range(len(df.columns)):
    feature = train_l.columns[i]
    unique_feature.append(train_l[feature].unique())
  summary['고윳값'] = unique_feature

  return summary

resumetable(train_l)


데이터셋 형상: (8405898, 12)


,피처,데이터 타입,결측값 개수,고윳값 개수,고윳값
0,id,object,<bound method IndexOpsMixin.value_counts of id...,2471,"[001519c8, 0022f953, 0042269b, 0059420b, 00758..."
1,event_id,int64,<bound method IndexOpsMixin.value_counts of id...,12876,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,down_time,int64,<bound method IndexOpsMixin.value_counts of id...,1836078,"[4526, 4558, 106571, 106686, 107196, 107296, 1..."
3,up_time,int64,<bound method IndexOpsMixin.value_counts of id...,1835993,"[4557, 4962, 106571, 106777, 107323, 107400, 1..."
4,action_time,int64,<bound method IndexOpsMixin.value_counts of id...,3509,"[31, 404, 0, 91, 127, 104, 107, 109, 138, 187,..."
5,activity,object,<bound method IndexOpsMixin.value_counts of id...,50,"[Nonproduction, Input, Remove/Cut, Replace, Mo..."
6,down_event,object,<bound method IndexOpsMixin.value_counts of id...,131,"[Leftclick, Shift, q, Space, Backspace, ., ,, ..."
7,up_event,object,<bound method IndexOpsMixin.value_counts of id...,130,"[Leftclick, Shift, q, Space, Backspace, ., ,, ..."
8,text_change,object,<bound method IndexOpsMixin.value_counts of id...,4111,"[NoChange, q, , ., ,, qqq qqqqq => , qqqqq..."
9,cursor_position,int64,<bound method IndexOpsMixin.value_counts of id...,7803,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
